# Calculate biogeographic traits

Created by 
* [Li, Chaonan (李超男)](https://www.researchgate.net/profile/Chaonan-Li-5) / licn@mtc.edu.cn / [Ecological Security and Protection Key Laboratory of Sichuan Province, Mianyang Normal University](https://zdsys.mtc.edu.cn/)
* Liao, Haijun (廖海君) / liaohj@mtc.edu.cn /
[Engineering Research Center of Chuanxibei RHS Construction at Mianyang Normal University of Sichuan Province](https://rhs.mtc.edu.cn/)

Reviewed by [Li, Xiangzhen (李香真)](https://www.researchgate.net/profile/Xiangzhen-Li-2) / lixz@fafu.edu.cn /
[College of Resources and Environment, Fujian Agriculture and Forestry University](https://zhxy.fafu.edu.cn/main.htm)

Our focus is on the spatial distribution of microbial traits, so we also need to calculate some microbial community measures. The [microgeo](https://github.com/ChaonanLi/microgeo) R package provides some basic functions for calculating microbial relative abundance, diversity, and community assembly mechanism indicators. Of course, you can also visualize more microbial traits onto maps, not limited to the traits calculated by the [microgeo](https://github.com/ChaonanLi/microgeo) package.

## Load required R packages

Here we need four R packages for this section of microgeo R package tutorial. Just run the following codes to import them into R environment.

In [1]:
# Install and load `magrittr`, `ggplot2`, `devtools` and `microgeo` packages 
if (!suppressMessages(require(magrittr))) install.packages("magrittr")
if (!require(ggplot2)  %>% suppressMessages) install.packages("ggplot2")
if (!require(devtools) %>% suppressMessages) install.packages("devtools")
if (!require(microgeo) %>% suppressMessages) devtools::install_github("ChaonanLi/microgeo")

## Create a standard microgeo dataset

We also need a standard microgeo dataset for the presentations in the section of tutorial.

In [2]:
# Example by using the map downloaded from DataV.GeoAtlas
data(qtp)
map <- read_aliyun_map(adcode = c(540000, 630000, 510000)) %>% suppressMessages() 
dataset.dts.aliyun <- create_dataset(mat = qtp$asv, ant = qtp$tax, met = qtp$met, map = map,
                                     phy = qtp$tre, env = qtp$env, lon = "longitude", lat = "latitude") 
dataset.dts.aliyun %<>% rarefy_count_table() 
dataset.dts.aliyun %<>% tidy_dataset()
dataset.dts.aliyun %>% show_dataset()

ℹ [2023-10-12 10:49:54] INFO ==> all samples fall within the map area!

ℹ [2023-10-12 10:49:54] INFO ==> dataset has been created successfully!

ℹ [2023-10-12 10:49:54] INFO ==> use `object %>% show_dataset()` to check the summary of dataset.

ℹ [2023-10-12 10:49:57] INFO ==> the ASV/gene abundance table has been rarefied with a sub-sample depth of 5310



── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## Calculate the relative abundance

We implemented a function named by `calc_rel_abund()`, which calculates the relative abundance of ASVs/genes at each of classification level. Just run the following codes.

In [3]:
# Calculate relative abundance
dataset.dts.aliyun %<>% calc_rel_abund() %>% suppressMessages()
head(dataset.dts.aliyun$abd$raw$Phylum[,1:5])

,p__,p__Acidobacteria,p__Actinobacteria,p__Armatimonadetes,p__Bacteroidetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s1,0.22598870,13.50282,49.26554,0.07532957,1.920904
s2,0.09416196,13.38983,51.46893,0.16949153,1.996234
s3,0.16949153,21.41243,42.90019,0.01883239,1.638418
s4,0.13182674,15.96987,46.93032,0.00000000,4.500942
s5,0.22598870,13.46516,54.42561,0.11299435,1.299435
s6,0.28248588,14.63277,47.06215,0.09416196,2.749529


In [4]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## Calculate the ecological markers

We also designed a function named by `calc_markers()`, which identifies the potentially ecological markers base on the relative abundance of ASV/genes at a given classification level. If there is only one environmnetal variable for the identification, then a correlation method would be used. If the are two or more environmental variables, then a mantel test would be applied. **Please note that such a function only works when the relative abundance of ASVs/genes are available!**. 

In [5]:
# Identify ecological markers based on soil pH in <env>[Correlation]
dataset.dts.aliyun %<>% calc_markers(use.var = 'pH', annotation.level = 'Phylum', r.thres = 0.1, use.dat = 'env') %>% suppressMessages()

In [6]:
# Show markers 
head(dataset.dts.aliyun$abd$mar$correlation)

,var,r,p
,<fct>,<dbl>,<dbl>
p__Acidobacteria,p__Acidobacteria,-0.3905858,1.311292e-46
p__Verrucomicrobia,p__Verrucomicrobia,-0.3586751,4.550495e-39
p__Rokubacteria,p__Rokubacteria,-0.3364620,2.647809e-34
p__Actinobacteria,p__Actinobacteria,0.3218787,2.218247e-31
p__Chloroflexi,p__Chloroflexi,0.2937423,3.522230e-26
p__Gemmatimonadetes,p__Gemmatimonadetes,0.2932926,4.220590e-26


In [7]:
# Identify ecological markers based on soil pH and TOC in <env> [Mantel test]
dataset.dts.aliyun %<>% calc_markers(use.var = c('pH', 'TOC'), annotation.level = 'Phylum', r.thres = 0.1, use.dat = 'env') %>% suppressMessages()

In [8]:
# Show markers 
head(dataset.dts.aliyun$abd$mar$correlation)

,var,r,p
,<fct>,<dbl>,<dbl>
p__Crenarchaeota,p__Crenarchaeota,0.2354545,0.001
p__Actinobacteria,p__Actinobacteria,0.1192869,0.001
p__Thaumarchaeota,p__Thaumarchaeota,0.1188555,0.001
p__Gemmatimonadetes,p__Gemmatimonadetes,0.1096113,0.001


In [9]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## Calculate alpha diversity indices

We implemented a functiobn named by `calc_alpha_div()`, which calculates serveral alpha diversity indices such as observed ASVs and Shannon index. Just run the following codes.

In [10]:
# Calculate alpha diversity indices
dataset.dts.aliyun %<>% calc_alpha_div(measures = c("observed", "shannon")) %>% suppressMessages()

In [11]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## Calculate beta diversity distance matrices

We implemented a function named by `calc_beta_div()`, which calculates serveral beta diversity distance matrix such as Bray-Curtis and Jarccard distance matrix. Just run the following codes.

In [12]:
# Calculate alpha diversity indices
dataset.dts.aliyun %<>% calc_beta_div(measures = c("bray", "jaccard")) %>% suppressMessages()

In [13]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


In [14]:
# Check results [Bray-Curtis distance]
dataset.dts.aliyun$div$bet$bray[1:4, 1:4]

,s1,s2,s3,s4
,<dbl>,<dbl>,<dbl>,<dbl>
s1,0.0000000,0.4851224,0.5772128,0.5018832
s2,0.4851224,0.0000000,0.4871940,0.4177024
s3,0.5772128,0.4871940,0.0000000,0.4013183
s4,0.5018832,0.4177024,0.4013183,0.0000000


## Calculate microbial community assembly indices

We implemeted a function named by `calc_phylo_asmb()`, which calculates the metrics involved in microbial community assembly. Just run the following codes.

In [15]:
# Calculate `alpha.phylo` null model
# runs = 9 just for an example. 999 runs may be better 
dataset.dts.aliyun %<>% calc_phylo_asmb(type = 'alpha.phylo', runs = 9, out.dir = '../test/calc_comm_asmb.rst') %>% suppressMessages()

In [16]:
# Check results 
head(dataset.dts.aliyun$asb$alpha.phylo)

,ntaxa,mntd.obs,mntd.rand.mean,mntd.rand.sd,mntd.obs.rank,mntd.obs.z,mntd.obs.p,runs
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s1,1002,0.04390328,0.08566004,0.006142561,1,-6.797941,0.1,9
s2,953,0.04841466,0.08458212,0.007260174,1,-4.981623,0.1,9
s3,840,0.04982293,0.09317449,0.008777077,1,-4.939179,0.1,9
s4,1023,0.04563238,0.08378930,0.005752973,1,-6.632557,0.1,9
s5,889,0.04766660,0.09208344,0.008908550,1,-4.985867,0.1,9
s6,977,0.04838227,0.08629231,0.006876969,1,-5.512609,0.1,9


In [17]:
# Calculate `beta.phylo` null model
# runs = 9 just for an example. 999 runs may be better 
dataset.dts.aliyun %<>% calc_phylo_asmb(type = 'beta.phylo', runs = 9, out.dir = '../test/calc_comm_asmb.rst') %>% suppressMessages()

In [18]:
# Check distance matrix 
names(dataset.dts.aliyun$asb$beta.phylo$dis) # distance matrices

[1] "b.mntd" "bc"     "rc"     "b.nti"

In [19]:
# Check the results 
head(dataset.dts.aliyun$asb$beta.phylo$raw$result) # the raw result of `iCAMP::qpen()`

,sample1,sample2,bMNTD,BC,bNTI,RC,process
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,s2,s1,0.01567063,0.4945386,-7.390820,-1.0000000,Homogeneous.Selection
2,s3,s1,0.01933495,0.5892655,-7.410212,-0.5555556,Homogeneous.Selection
3,s4,s1,0.01908942,0.5000000,-1.928595,-1.0000000,Homogenizing.Dispersal
4,s5,s1,0.01572110,0.4173258,-2.251657,-1.0000000,Homogeneous.Selection
5,s6,s1,0.01578337,0.4721281,-3.574340,-1.0000000,Homogeneous.Selection
6,s7,s1,0.01845051,0.4934087,-5.350482,-1.0000000,Homogeneous.Selection


In [20]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)

✔ object$asb$beta.phylo$raw$result: 7 beta phylogenetic assembling index/indices (sample1, sample2, bMNTD, ...)

✔ object$asb$beta.phylo$dis: 4 beta phylogenetic distance matrix/matrices (b.mntd, bc, rc, b.nti)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`
